In [ ]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [ ]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

In [ ]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

In [ ]:
NUMBER_POST = 10
post_list = []
reactor_list = []
total_reactor_real = 0
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_react.txt')

In [ ]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

In [ ]:
def extract_reactor(reactors, post_id):
    cnt_reactor = 0
    global reactor_list
    for reactor in reactors:
        reactor['post_id'] = post_id
        reactor_list.append(reactor)
        cnt_reactor += 1
        time.sleep(np.random.uniform(0.5, 2.5))
    return cnt_reactor

In [ ]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "reactors": True,
            "reactors": "generator",
            "reactions": True
        },
        timeout=120,
    ):
        print(post)
        post_list.append(post)
        num_sharer = extract_reactor(post['reactors'], post['original_request_url'])
        print(f'Extracted {num_sharer}/{post["reaction_count"]} reactors')
        total_reactor_real += post['reaction_count']
        utils.sleep(np.random.randint(60, 180))
        print()
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

In [ ]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(reactor_list)}/{total_reactor_real}")

In [ ]:
if reactor_list:
    reactor_df = utils.save_data(
        data_list=reactor_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="reactors"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)